In [1]:
import pandas as pd 
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer, util
import pickle 

df = pd.read_csv("data/parsed_docs.csv")
model = SentenceTransformer('msmarco-distilbert-base-v3')

idx = AnnoyIndex(768, "angular")
idx.load("saved_indices/msmarco-distilbert-base-idx.ann", True)


def cosine_similarity_transform(angular_distance):
    return (2-(angular_distance**2))/2

In [2]:
from utils import extract_topics_from_file
topics = extract_topics_from_file("qrels/2020/topics_test.txt")

titles = []
narratives = []
numbers = []
for topic in topics:
    titles.append(topic["title"])
    narratives.append(topic["narrative"])
    numbers.append(topic["number"])

titles_embed = model.encode(titles, show_progress_bar=True)
narratives_embed = model.encode(narratives, show_progress_bar=True)

In [6]:
def extract_annoy_results(docs, scores):
    results = {}
    for i, doc_id in enumerate(docs):
        docno = df.iloc[doc_id].docno
        score = cosine_similarity_transform(scores[i])
        results[docno]=score    
    return results

In [11]:
title_run = {}
narrative_run = {}

for i, num in enumerate(numbers):

    te = titles_embed[i]
    doc_idx, scores = idx.get_nns_by_vector(te, 50, search_k=-1, include_distances=True)
    te_results = extract_annoy_results(doc_idx, scores)

    title_run[num] = te_results

    ne = narratives_embed[i]
    doc_idx, scores = idx.get_nns_by_vector(ne, 50, search_k=-1, include_distances=True)
    ne_results = extract_annoy_results(doc_idx, scores)

    title_run[num] = te_results
    narrative_run[num] = ne_results

In [13]:
import pytrec_eval
from utils import read_qrel_from_file, evaluate_run
import numpy as np 
import json 

qrel = read_qrel_from_file("qrels/2020/cair2020_qrel.txt")

metrics: set = {'map', 'ndcg', 'P_5'}
evaluator = pytrec_eval.RelevanceEvaluator(qrel, metrics)

In [17]:
run_results = evaluator.evaluate(title_run)

map_scores = [v["map"] for k,v in run_results.items()]
p_scores  = [v["P_5"] for k,v in run_results.items()]

display(run_results)

print("Aggregate results")
print("Average MAP: ", np.mean(map_scores))
print("Average P_5: ", np.mean(p_scores))

with open("results/msmarco-distilbert-base-v3-baseline/title-results.txt", "w") as f:
    f.write(f"Average MAP: {np.mean(map_scores)}\n")
    f.write(f"Average P_5: {np.mean(p_scores)}\n")
    f.write(f"Individual run results: \n")
    f.write(json.dumps(run_results, indent=4))

with open("results/msmarco-distilbert-base-v3-baseline/title-run.txt", "w") as f:
    f.write(json.dumps(title_run, indent=4))

{'1': {'map': 0.028186453348101336, 'P_5': 0.2, 'ndcg': 0.13783416795111758},
 '2': {'map': 0.0878333217103761, 'P_5': 0.2, 'ndcg': 0.25471330971680556},
 '3': {'map': 0.2902547046753907, 'P_5': 0.6, 'ndcg': 0.5540259522972194},
 '4': {'map': 0.18391645772620296, 'P_5': 0.6, 'ndcg': 0.3997304140576192},
 '5': {'map': 0.18594354984581526, 'P_5': 0.8, 'ndcg': 0.38494170748253004},
 '6': {'map': 0.18616409625044408, 'P_5': 0.2, 'ndcg': 0.4089745464735173},
 '7': {'map': 0.13881781553342115, 'P_5': 0.4, 'ndcg': 0.3225321900572252},
 '8': {'map': 0.3177777777777777, 'P_5': 0.6, 'ndcg': 0.5244510491500967},
 '9': {'map': 0.028128054740957965, 'P_5': 0.2, 'ndcg': 0.14588739561224537},
 '10': {'map': 0.24266395154553047, 'P_5': 0.8, 'ndcg': 0.47083199301647755},
 '11': {'map': 0.021755701458440323, 'P_5': 0.2, 'ndcg': 0.11813375280458877},
 '12': {'map': 0.15696248196248197, 'P_5': 0.8, 'ndcg': 0.32662179994368323},
 '13': {'map': 0.15455119195056582, 'P_5': 0.6, 'ndcg': 0.3579751230710474},
 

Aggregate results
Average MAP:  0.16110136097152256
Average P_5:  0.5


In [18]:
run_results = evaluator.evaluate(narrative_run)

map_scores = [v["map"] for k,v in run_results.items()]
p_scores  = [v["P_5"] for k,v in run_results.items()]

display(run_results)

print("Aggregate results")
print("Average MAP: ", np.mean(map_scores))
print("Average P_5: ", np.mean(p_scores))

with open("results/msmarco-distilbert-base-v3-baseline/narrative-results.txt", "w") as f:
    f.write(f"Average MAP: {np.mean(map_scores)}\n")
    f.write(f"Average P_5: {np.mean(p_scores)}\n")
    f.write(f"Individual run results: \n")
    f.write(json.dumps(run_results, indent=4))

with open("results/msmarco-distilbert-base-v3-baseline/narrative-title-run.txt", "w") as f:
    f.write(json.dumps(title_run, indent=4))

{'1': {'map': 0.0944483692782852, 'P_5': 0.6, 'ndcg': 0.253528433793341},
 '2': {'map': 0.10162770346428517, 'P_5': 0.4, 'ndcg': 0.29225178246877154},
 '3': {'map': 0.5101200047939178, 'P_5': 0.8, 'ndcg': 0.7434936642100887},
 '4': {'map': 0.20869065652915597, 'P_5': 0.2, 'ndcg': 0.4172871060592224},
 '5': {'map': 0.2928946469906547, 'P_5': 1.0, 'ndcg': 0.4734746258813843},
 '6': {'map': 0.17505825967732563, 'P_5': 0.4, 'ndcg': 0.4350588997889799},
 '7': {'map': 0.14101823724755508, 'P_5': 0.6, 'ndcg': 0.32631342800728835},
 '8': {'map': 0.0, 'P_5': 0.0, 'ndcg': 0.0},
 '9': {'map': 0.13411534701857283, 'P_5': 0.4, 'ndcg': 0.34339786214378665},
 '10': {'map': 0.32765752765752765, 'P_5': 0.8, 'ndcg': 0.5511097322857154},
 '11': {'map': 0.020416873957678213, 'P_5': 0.2, 'ndcg': 0.11501546574962641},
 '12': {'map': 0.29489544618084224, 'P_5': 0.8, 'ndcg': 0.5165269941438778},
 '13': {'map': 0.3049577486819272, 'P_5': 0.8, 'ndcg': 0.5177103649165715},
 '14': {'map': 0.06140350877192983, 'P_

Aggregate results
Average MAP:  0.22390723403389456
Average P_5:  0.55
